In [1]:
import os
import pandas as pd
import numpy as np
from project import Project 

ant_project = Project('ANT')
ant_project.set_train_test()
ant_project.prepare_data()



In [2]:
ant_project.train_global_model('SVM')
ant_svm = ant_project.models['SVM']


In [3]:
import warnings
warnings.filterwarnings('ignore')

ant_project.train_global_model('LR')
ant_project.train_global_model('BRCG')
ant_project.train_global_model('LogRR')

Learning CNF rule with complexity parameters lambda0=0.001, lambda1=0.001
Initial LP solved
Iteration: 1, Objective: 0.2058
Iteration: 2, Objective: 0.1862
Iteration: 3, Objective: 0.1702
Iteration: 4, Objective: 0.1566
Iteration: 5, Objective: 0.1406
Iteration: 6, Objective: 0.1340
Iteration: 7, Objective: 0.1326
Iteration: 8, Objective: 0.1308
Iteration: 9, Objective: 0.1159
Iteration: 10, Objective: 0.1094
Iteration: 11, Objective: 0.1076
Iteration: 12, Objective: 0.1068
Iteration: 13, Objective: 0.1068
Iteration: 14, Objective: 0.1065
Iteration: 15, Objective: 0.1055
Iteration: 16, Objective: 0.1051
Iteration: 17, Objective: 0.1050
Iteration: 18, Objective: 0.1047
Iteration: 19, Objective: 0.1047


In [4]:
ant_lr = ant_project.models['LR']
ant_brcg = ant_project.models['BRCG']
ant_lrr = ant_project.models['LogRR']

In [5]:
ant_pred_svm = ant_svm.predict(ant_project.X_testStd)
ant_defect_prob_svm = ant_svm.predict_proba(ant_project.X_testStd)[:,1]

ant_pred_lr = ant_lr.predict(ant_project.X_testStd)
ant_defect_prob_lr = ant_lr.predict_proba(ant_project.X_testStd)[:,1]

ant_pred_brcg = ant_brcg.predict(ant_project.X_test_bin)>0

ant_pred_lrr = ant_lrr.predict(ant_project.X_test_bin, ant_project.X_testStd)
ant_defect_prob_lrr = ant_lrr.predict_proba(ant_project.X_test_bin, ant_project.X_testStd)

cnf


In [6]:
from eval_functions import *
evaluate_model_performance(ant_project,'SVM')
evaluate_model_performance(ant_project,'LR')
evaluate_model_performance(ant_project,'BRCG')
evaluate_model_performance(ant_project,'LogRR')

Dataset project: ANT
SVM Training accuracy(in %): 84.16988416988417
SVM Test accuracy(in %): 74.49664429530202
SVM precision(in %): 45.0
SVM recall(in %): 65.06024096385542
SVM f1(in %): 53.201970443349744
Dataset project: ANT
LR Training accuracy(in %): 81.66023166023166
LR Test accuracy(in %): 74.09395973154362
LR precision(in %): 44.10480349344978
LR recall(in %): 60.8433734939759
LR f1(in %): 51.13924050632911
Dataset project: ANT
cnf
cnf
BRCG Training accuracy(in %): 91.6988416988417
BRCG Test accuracy(in %): 77.58389261744966
BRCG precision(in %): 49.746192893401016
BRCG recall(in %): 59.036144578313255
BRCG f1(in %): 53.99449035812673
Dataset project: ANT
LogRR Training accuracy(in %): 92.85714285714286
LogRR Test accuracy(in %): 75.83892617449665
LogRR precision(in %): 46.464646464646464
LogRR recall(in %): 55.42168674698795
LogRR f1(in %): 50.54945054945055


In [14]:
# BR explaination
exp_BRCG = ant_brcg.explain(maxConj=5) #max 5 rules
print(len(exp_BRCG['rules']))
print(exp_BRCG)

5
{'isCNF': True, 'rules': ['loc <= 23.70', 'wmc > 5.00 AND loc <= 169.00', 'mfa <= 0.00 AND cam > 0.50', 'lcom > 4.00 AND dam <= 0.00', 'dit > 1.00 AND ce <= 7.00 AND loc <= 355.40']}


In [ ]:
# LogRR explanation
exp_LRR = ant_lrr.explain()
exp_LRRh = ant_lrr.explain(highDegOnly=True)

print(exp_LRR)

# Separate 1st-degree rules into (feature, operation, value) to count unique features
dfx2 = exp_LRR['rule/numerical feature']
highOrd = exp_LRRh['rule']
dfx2 = dfx2[~dfx2.isin(highOrd)].str.split(' ', expand=True)
dfx2.columns = ['feature','operation','value']
uniq = dfx2['feature'].unique() #features that contribute to predictions

train_data, test_data = ant_project.get_datasets()
data = pd.concat([train_data,test_data])
ant_lrr.visualize(data,ant_project.featureBinarizer,['mfa'])
ant_lrr.visualize(data,ant_project.featureBinarizer,['amc'])
ant_lrr.visualize(data,ant_project.featureBinarizer,['dit'])



## CEM Explainer (Contrastive Explanation)
Contrastive explanations (only accepts KerasClassifier models)
- provide information about what minimal changes to the code would have changed the decision of the AI model from defective to non-defective or vice-versa (pertinent negatives).
- also highlights a minimal set of features and their values that would still maintain the original decision
-to understand trends in the behaviour of AI models in predicting if code is defective in the investigated class
- Looking at aggregate statistics of pertinent negatives, we can get insight into what minimal set of features and their values play an important role in predicting defect

Computing Pertinent Negatives (PN)
- In order to compute pertinent negatives, the CEM explainer computes a set of code metrics that is close to the original class but for which the prediction is different.

In [1]:
import os
import pandas as pd
import numpy as np
from project import Project 

ant_project = Project('ANT')
ant_project.set_train_test()
ant_project.prepare_data()

In [2]:
ant_project.train_global_model('NN')
from eval_functions import *
evaluate_model_performance(ant_project,'NN')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 2)                 22        
                                                                 
Total params: 232
Trainable params: 232
Non-trainable params: 0
_________________________________________________________________
Train on 518 samples
Epoch 1/500
518/518 [==============================] - 0s 460us/sample - loss: 0.6931 - accuracy: 0.8301
Epoch 2/500
518/518 [==============================] - 0s 25us/sample - loss: 0.6931 - accuracy: 0.7876
Epoch 3/500
518/518 [==============================] - 0s 25us/sample - loss: 0.6931 - accuracy: 0.6834
Epoch 4/500
518/518 [==============================] - 0s 26us/sample - loss: 0.6931 - accuracy: 0.5502
Epoch 5/500
518/

C:\Users\swen\anaconda3\envs\PyExplainer\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


In [3]:
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Dense
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from aix360.algorithms.contrastive import CEMExplainer, KerasClassifier
from aix360.algorithms.protodash import ProtodashExplainer

In [5]:
train_data, test_data = ant_project.get_datasets()
data = pd.concat([train_data,test_data])
X_testNorm = ant_project.X_testNorm
idx ='org.apache.tools.ant.taskdefs.WaitFor'
y = ant_project.X_test.loc[idx,:].to_numpy().reshape((1,) + ant_project.X_test.loc[idx,:].to_numpy().shape)
x = X_testNorm[4].reshape((1,) + X_testNorm[4].shape)
nn= ant_project.models['NN']
class_names = ['clean','defect']
# y = test_data.loc[idx,:].to_numpy()
# idx = 4
# X = y.reshape((1,) + y.shape)
print("Computing PN for Sample:", idx)
print("Prediction made by the model:", nn.predict(x))
print("Prediction probabilities:", class_names[np.argmax(nn.predict(x))])

mymodel = KerasClassifier(nn)
explainer = CEMExplainer(mymodel)

arg_mode = 'PN' # Find pertinent negatives
arg_max_iter = 1000 # Maximum number of iterations to search for the optimal PN for given parameter settings
arg_init_const = 10.0 # Initial coefficient value for main loss term that encourages class change
arg_b = 9 # No. of updates to the coefficient of the main loss term
arg_kappa = 0.2 # Minimum confidence gap between the PNs (changed) class probability and original class' probability
arg_beta = 1e-1 # Controls sparsity of the solution (L1 loss)
arg_gamma = 100 # Controls how much to adhere to a (optionally trained) auto-encoder
my_AE_model = None # Pointer to an auto-encoder
arg_alpha = 0.01 # Penalizes L2 norm of the solution
arg_threshold = 1. # Automatically turn off features <= arg_threshold if arg_threshold < 1
arg_offset = 0.5 # the model assumes classifier trained on data normalized
                # in [-arg_offset, arg_offset] range, where arg_offset is 0 or 0.5

# Find PN for applicant 1272
(adv_pn, delta_pn, info_pn) = explainer.explain_instance(x, arg_mode, my_AE_model, arg_kappa, arg_b,arg_max_iter, arg_init_const, arg_beta, arg_gamma,arg_alpha,arg_threshold,arg_offset)

Computing PN for Sample: org.apache.tools.ant.taskdefs.WaitFor
Prediction made by the model: [[19.874956 19.876747]]
Prediction probabilities: defect


C:\Users\swen\anaconda3\envs\PyExplainer\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


iter:0 const:[10.]
Loss_Overall:2.0179, Loss_Attack:2.0179
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:19.8767, max_nontarget_lab_score:19.8750

iter:500 const:[10.]
Loss_Overall:2.0179, Loss_Attack:2.0179
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:19.8767, max_nontarget_lab_score:19.8750

iter:0 const:[100.]
Loss_Overall:20.1791, Loss_Attack:20.1791
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:19.8767, max_nontarget_lab_score:19.8750

iter:500 const:[100.]
Loss_Overall:20.1791, Loss_Attack:20.1791
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:19.8767, max_nontarget_lab_score:19.8750

iter:0 const:[1000.]
Loss_Overall:201.7910, Loss_Attack:201.7910
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:19.8767, max_nontarget_lab_score:19.8750

iter:500 const:[1000.]
Loss_Overall:201.7910, Loss_Attack:201.7910
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_scor

In [6]:
# should be moved 
Z = np.vstack((ant_project.X_train_rs, ant_project.X_test))
Zmax = np.max(Z, axis=0)
Zmin = np.min(Z, axis=0)
# rescale a sample to recover original values for normalized values. 
def rescale(X):
    return(np.multiply ( X + 0.5, (Zmax - Zmin) ) + Zmin)

In [7]:

Xpn = adv_pn
classes = [ class_names[np.argmax(nn.predict(x))], class_names[np.argmax(nn.predict(Xpn))], 'NIL' ]

print("Sample:", idx)
print("prediction(X)", nn.predict(x), class_names[np.argmax(nn.predict(x))])
print("prediction(Xpn)", nn.predict(Xpn), class_names[np.argmax(nn.predict(Xpn))] )

X_re = rescale(x) # Convert values back to original scale from normalized
Xpn_re = rescale(Xpn)
Xpn_re = np.around(Xpn_re.astype(np.double), 2)

delta_re = Xpn_re - X_re
delta_re = np.around(delta_re.astype(np.double), 2)
delta_re[np.absolute(delta_re) < 1e-4] = 0

X3 = np.vstack((X_re, Xpn_re, delta_re))

dfre = pd.DataFrame.from_records(X3) # Create dataframe to display original point, PN and difference (delta)
dfre[23] = classes

dfre.columns = data.columns
dfre.rename(index={0:'X',1:'X_PN', 2:'(X_PN - X)'}, inplace=True)
dfret = dfre.transpose()

def highlight_ce(s, col, ncols):
    if (type(s[col]) != str):
        if (s[col] > 0):
            return(['background-color: yellow']*ncols)    
    return(['background-color: white']*ncols)

dfret.style.apply(highlight_ce, col='(X_PN - X)', ncols=3, axis=1)

Sample: org.apache.tools.ant.taskdefs.WaitFor
prediction(X) [[19.874956 19.876747]] defect
prediction(Xpn) [[46.293064 46.293743]] defect


,X,X_PN,(X_PN - X)
wmc,9.000000,0.000000,-9.000000
dit,3.000000,1.000000,-2.000000
noc,0.000000,0.000000,0.000000
cbo,5.000000,0.000000,-5.000000
rfc,26.000000,0.000000,-26.000000
lcom,16.000000,0.000000,-16.000000
ca,0.000000,0.000000,0.000000
ce,5.000000,0.000000,-5.000000
npm,7.000000,0.000000,-7.000000
lcomt,0.750000,0.000000,-0.750000
